In [1]:
from PIL import Image
from tensorflow.keras.layers import Dense,Input
from keras.layers import Input , Lambda , Flatten , Dense , Dropout
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.applications import MobileNet
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np 
import glob as glob
datagen = ImageDataGenerator()
import matplotlib.pyplot as plt

import json
import random
import cv2

import tensorflow as tf

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = [224, 224]

train_photo = 'train_image/'
test_photo = 'test_image/'
val_photo = 'val_image/'

In [3]:
myvgg = VGG16(input_shape=IMAGE_SIZE +  [3], weights='imagenet', include_top=False)

In [4]:
# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [5]:
folder = glob.glob('Desktop\task4-mlops\train_image')

In [6]:
def Topmodel(bottom_model, num_classes, D=256):
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")( top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")( top_model)
    return top_model

myvgg.input
myvgg.summary

<bound method Network.summary of <keras.engine.training.Model object at 0x0000020B30B7C8C8>>

In [7]:
# Set our class number to 2
num_classes = 2

FC_Head = Topmodel(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [9]:
train_set = train_datagen.flow_from_directory('C:/Users/Dell/Desktop/task4-mlops/train_image/',
                                              target_size = (224, 224),
                                              batch_size = 32,
                                              class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('C:/Users/Dell/Desktop/task4-mlops/test_image/',
                                              target_size = (224, 224),
                                              batch_size = 32,
                                              class_mode = 'categorical')

Found 1200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [13]:
checkpoint = ModelCheckpoint("facerecognition_model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.2,
                              patience = 3, 
                              verbose = 1,
                              min_delta = 0.0001)

callbacks = [earlystop, checkpoint,reduce_lr]
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 125
nb_validation_samples = 25

epochs = 5
batch_size = 8

history = model.fit_generator(
    train_set,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_set,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
15/15 [==============================] - 67s 4s/step - loss: 0.6933 - accuracy: 0.4612 - val_loss: 0.6928 - val_accuracy: 0.5729

Epoch 00001: val_loss improved from inf to 0.69282, saving model to facerecognition_model.h5
Epoch 2/5
15/15 [==============================] - 65s 4s/step - loss: 0.6932 - accuracy: 0.4375 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00002: val_loss did not improve from 0.69282
Epoch 3/5
15/15 [==============================] - 65s 4s/step - loss: 0.6931 - accuracy: 0.5167 - val_loss: 0.6930 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 0.69282
Epoch 4/5
15/15 [==============================] - 65s 4s/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6933 - val_accuracy: 0.5625

Epoch 00004: val_loss did not improve from 0.69282

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 5/5
15/15 [==============================] - 64s 4s/step - loss: 0.6933 - accuracy: 0.4828 - val_loss: 0.6